# Store mechanosorptive results into Excel files
This notebook store the results of mechanosorptive experiments into Excel files organized by sample type. Each file contains multiple sheets (one per experiment), and each sheet reports detailed sample information (dimensions, applied load, loading degree) along with time, creep strain, creep compliance, and moisture measurements.


### 1) Import libraries and set paths

In [ ]:
import os
import ast
from mat73 import loadmat
from math import *
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
import glob
from datetime import datetime, timedelta
from collections import defaultdict
from scipy.optimize import lsq_linear
from scipy.optimize import least_squares
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import make_interp_spline
from scipy.stats import trim_mean
from sklearn.metrics import r2_score
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import re
from collections import defaultdict
np.set_printoptions(threshold=np.inf)

folder_path = '/home/aferrara/Desktop/Creep_experiments_1/'
savepath = '/home/aferrara/Desktop/creep-evaluation-routine/ms_creep_python_routine/MS_Creep_Tests_Dataset'
# Set file paths
dvs_path = savepath + '/dvs_data.npz'
elastic_path = savepath + 'elastic_compliances_Ferrara_Wittel_2024.csv'
viscoel_path = savepath + 'master_vec_prony_param.csv'

### 2) Customized functions

In [ ]:
# Retrieve experiment path
def get_rawdata_path(exp_code, main_path):
    # Construct experiment path
    exp_path = os.path.join(main_path, exp_code)
    # Check if folder exists
    if not os.path.isdir(exp_path): raise FileNotFoundError(f"There is no folder for the experiment ({exp_path}).")
    # Check if the folder is empty
    if not os.listdir(exp_path): raise FileNotFoundError(f"The folder for the experiment exists but is empty ({exp_path}).")
    return exp_path
# end: get_rawdata_path

# Retrieve experiment data
def get_experiment_data(exp_code, main_path):

    # Get experiment folder path
    exp_path = get_rawdata_path(exp_code, main_path)
    data_path = os.path.join(exp_path, f"{exp_code}_Data_new.txt")

    # Check if file exists
    if not os.path.isfile(data_path): raise FileNotFoundError(f"Experiment data file '{data_path}' not found.")

    # Read the data file into a Pandas DataFrame
    df = pd.read_csv(data_path, delimiter='\t')  # adjust delimiter if needed

    # Convert DataFrame rows to a list of dictionaries
    samples_data = []
    for _, row in df.iterrows():
        sample = {
            "sample_holder": row[0],    # Column 1
            "exp_name": row[2],         # Column 3
            "img_name": row[3],         # Column 4
            "thick": row[10],           # Column 11
            "width": row[11],           # Column 12
            "area": row[10] * row[11],  # Column 11 * Column 12
            "load": row[6],             # Column 7
            "failure_load": row[7],     # Column 8
            "initial_load": row[8],     # Column 9
            "load_cell": row[9],        # Column 10
            "RH": row[12]               # Column 13
        }
        samples_data.append(sample)

    return samples_data, exp_path
# end: def get_experiment_data

# Retrieve all unique sample types across experiments
def get_sample_types(main_path, prefix="MST30-90"):
    folders = sorted(
        [d for d in os.listdir(main_path)
         if os.path.isdir(os.path.join(main_path, d)) and d.startswith(prefix)])
    types = set()
    for exp in folders:
        samples_data, _ = get_experiment_data(exp, main_path)
        for sample in samples_data:
            st = sample['exp_name'][3:5]
            if st == 'LT':
                tissue = "EW" if int(sample['exp_name'][-1]) >= 4 else "LW"
                st = f"LT-{tissue}"
            types.add(st)
    return sorted(types)
# end: def get_sample_types

# Load strain results of DIC analysis
def load_strain_struct(grey_folder):
    # Find the first matching .mat file
    export_file_strain = glob.glob(os.path.join(grey_folder, '*_strain_DIC.mat'))[0]
    if not os.path.isfile(export_file_strain):
        raise FileNotFoundError(f"The data struct {os.path.basename(export_file_strain)} does not exist in {grey_folder}.")
    # Load mat file
    export_strain = loadmat(export_file_strain)['new_struct']
    # Convert MATLAB serial datenum to Python datetime 
    time_name = ['imgtime', 'loadtime', 'RHtime']
    for name in time_name:
        matlab_datenum = export_strain[name]  # MATLAB serial number
        if isinstance(matlab_datenum, np.ndarray):  # If it's an array, process each element
            export_strain[name] = np.array([
                (datetime.fromordinal(int(d)) + timedelta(days=d % 1) - timedelta(days=366)).replace(microsecond=0) for d in matlab_datenum])
        else:  # If it's a single value, process directly
            export_strain[name] = (datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum % 1) - timedelta(days=366)).replace(microsecond=0)

    return export_strain
# end: def load_strain_struct

# Locate previous and next index of drop 
def locate_drop_idx(cycle_idx, drop_idx, threshold=3):

    # Compute distances to each cycle boundary
    distances = np.abs(cycle_idx - drop_idx)
    min_dist = distances.min()

    if min_dist <= threshold:
        # If drop too close, snap to the nearest cycle boundary
        nearest = cycle_idx[np.argmin(distances)]
        drop_idx = nearest
        # Only look for the cycle boundary before drop
        before = cycle_idx[cycle_idx < drop_idx]
        before_idx = before.max() if before.size else None
        after_idx = drop_idx
    else:
        # If drop is not too close, find both before and after
        before = cycle_idx[cycle_idx < drop_idx]
        after  = cycle_idx[cycle_idx > drop_idx]
        before_idx = before.max() if before.size else None
        after_idx  = after.min() if after.size  else None

    return before_idx, drop_idx, after_idx
# end: locate_drop_idx

# Collect all samples matching a given sample type
def get_samples_by_type(sample_type, main_path, prefix="MST30-90"):
    
    matches = []
    folders = sorted(
        [d for d in os.listdir(main_path)
         if os.path.isdir(os.path.join(main_path, d)) and d.startswith(prefix)])

    for exp in folders:
        samples_data, exp_path = get_experiment_data(exp, main_path)
        for s in samples_data[::2]:
            st = s['exp_name'][3:5]
            if st == 'LT':
                tissue = "EW" if int(s['exp_name'][-1]) >= 4 else "LW"
                st = f"LT-{tissue}"
            if st == sample_type:
                matches.append((exp, exp_path, f"{s['exp_name']}_{s['sample_holder']}"))
                
    return matches
# end: get_samples_by_type

# Estract loading degree of a given sample
def extract_loading_deg(exp_path, sample_name):

    # Set path to image folder
    file_folder = os.path.join(exp_path, "Results", f"{sample_name}")
    # Check if folder exists
    if not os.path.isdir(file_folder): raise FileNotFoundError(f"The folder {file_folder} does not exist.")
    # Load data
    strain_results = load_strain_struct(file_folder)

    return strain_results['loading_deg']
# end: def extract_loading_deg

# Find indexes of local stress drop
def find_local_drop(arr, stress_drop=None, window=5):

    # Compute all differences
    arr = np.asarray(arr)
    n = arr.size
    if n < 2: raise ValueError("Array must have at least 2 elements to find a drop.")
    diffs = np.diff(arr)
    
    if stress_drop is None:
        # If no guess, then global search
        return int(np.argmin(diffs)) + 1

    # Clamp stress_drop to valid [0, n-1]
    sd = int(np.clip(stress_drop, 0, n - 1))
    # Convert to diffs‐index domain: diffs[j] = arr[j+1] - arr[j]
    # so arr‐index i corresponds to diffs‐index j = i-1
    j_center = sd - 1

    # Define window in diffs‐space
    j0 = max(0, j_center - window)
    j1 = min(n - 2, j_center + window)

    # Find drop
    local = diffs[j0:j1+1]
    if local.size:
        j_drop = j0 + int(np.argmin(local))
        return j_drop + 1

    # If for some reason the window was empty,
    # fall back to global search
    return int(np.argmin(diffs)) + 1
# end: def find_local_drop

# Build identical moisture cycles with corresponding time and stress
def build_cycles(tdata, wdata, total_cycles, stress_value, stressed_cycles):

    # Convert to arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    if tdata.shape != wdata.shape: raise ValueError("tdata and wdata must have the same shape")

    # Determine cycle period
    period = tdata[-1] - tdata[0] + tdata[-1] - tdata[-2]

    # Pre-allocate arrays
    n_per_cycle = tdata.size
    N = n_per_cycle * total_cycles
    t_full = np.empty(N, dtype=tdata.dtype)
    w_full = np.empty(N, dtype=wdata.dtype)
    s_full = np.empty(N, dtype=float)

    for cycle in range(total_cycles):
        start = cycle * n_per_cycle
        end = start + n_per_cycle

        # Time offset
        t_full[start:end] = tdata + cycle * period
        w_full[start:end] = wdata

        # Apply stress for initial cycles, else 0
        if cycle < stressed_cycles:
            s_full[start:end] = stress_value
        elif cycle == stressed_cycles:
            s_full[start-1] = 0.0
            s_full[start:end] = 0.0
        else:
            s_full[start:end] = 0.0

    return t_full, w_full, s_full
# end: build_cycles

# Build moisture cycles with different first sorption with corresponding time and stress
def build_cycles_with_initial(tdata0, wdata0, tdata, wdata, total_cycles, stress_value, stressed_cycles):
    
    # Convert to arrays
    t0 = np.asarray(tdata0)
    w0 = np.asarray(wdata0)
    t  = np.asarray(tdata)
    w  = np.asarray(wdata)

    # Determine cycle period
    dt = t[-1] - t[-2]
    period = (t[-1] - t[0]) + dt

    # Total length
    n0 = t0.size
    n  = t.size
    N  = n0 + n*(total_cycles-1)
    # Pre-allocate
    t_full = np.empty(N, dtype=t.dtype)
    w_full = np.empty(N, dtype=w.dtype)
    s_full = np.empty(N, dtype=float)

    # Set first cycle
    t_full[:n0] = t0
    w_full[:n0] = w0
    s_full[:n0] = stress_value if 0 < stressed_cycles else 0.0
    # Set remaining cycles
    for i in range(1, total_cycles):
        start = n0 + (i-1)*n
        end   = start + n
        # Time shift
        t_full[start:end] = t + (i-1)*period
        w_full[start:end] = w
        # Apply stress for initial cycles, else 0
        if i < stressed_cycles:
            s_full[start:end] = stress_value
        elif i == stressed_cycles:
            s_full[start-1] = 0.0
            s_full[start:end] = 0.0
        else:
            s_full[start:end] = 0.0

    return t_full, w_full, s_full
# end: build_cycles_with_initial

# Build moisture cycles with different first sorption from dvs data with corresponding time and stress
def build_dynamic_cycles(tdata0, wdata0, tdata, wdata, t_exp, cycle_idx, idx_before, idx_drop, stress_val):
    
    # Inititalize arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    if tdata.shape != wdata.shape: raise ValueError("tdata and wdata must have the same shape")
    
    # Create loading cycles with first sorption different
    n_pre = np.searchsorted(cycle_idx, idx_before,  side="right") + 1 if len(stress_val) > 1 else np.searchsorted(cycle_idx, idx_before,  side="right")
    t_full, w_full, s_full = build_cycles_with_initial(tdata0, wdata0, tdata, wdata, n_pre, stress_val[0], n_pre)

    if t_full[-1] > t_exp[idx_drop]:
        idx_closest = np.abs(t_full - t_exp[idx_drop-2]).argmin()
        s_full = s_full[:idx_closest+1]
        t_full = t_full[:idx_closest+1]
        w_full = w_full[:idx_closest+1]

    return t_full, w_full, s_full
# end: build_dynamic_cycles

# Find end index of each moisture cycle
def find_cycle_ends(w, tol=1e-5, merge_gap=5):
    w = np.asarray(w)
    w_min = w.min()

    # Find all 'low plateau' indices
    low_idxs = np.where(w <= (w_min + tol))[0]
    if low_idxs.size == 0:
        return []

    # Cluster them by index proximity
    clusters = [[low_idxs[0]]]
    for idx in low_idxs[1:]:
        if idx - clusters[-1][-1] <= merge_gap:
            # Same plateau cluster
            clusters[-1].append(idx)
        else:
            # New plateau cluster
            clusters.append([idx])

    # Pick the largest index from each cluster
    cycle_end_indices = [max(cluster) for cluster in clusters]

    return cycle_end_indices
# end: def find_cycle_ends

# Convert RH into mean w (mean S/D of tissues from standard dvs test)
def RH_to_w_mean(x):
    w = 7.155e-11 * x**5 - 1.659e-08 * x**4 + 1.75e-06 * x**3 - 9.343e-05 * x**2 + 0.003795 * x + 0.002295
    return w
# end: RH_to_w_mean

# Calculate hygroexpansion strain
def calculate_hygroexp_strain(sample_type, wdata):

    # Set hygroexpansion coefficients
    alpha = {"R": 0.182,
            "T": 0.343,
            "L": 0.0061}
    # Extract loading direction
    long_dir = sample_type[0]
    # Calculate hygroexpansion strain
    eps_w = alpha[long_dir]*(wdata-wdata[0])

    return eps_w
# end: def calculate_hygroexp_strain

# Fit elastic compliances to moisture
def fit_elastic_comp(elastic_path):

    # Read csv file
    df = pd.read_csv(elastic_path)
    # Group and compute mean compliance
    mean_df = (df.groupby(['sample_type', 'RH'], as_index=False).C0.mean().rename(columns={'C0': 'C0_mean'}))
    # Calculate average w for each RH
    mean_df['w'] = RH_to_w_mean(mean_df['RH'])
    # Fit quadratic function in w
    el_fits = {}
    for stype, sub in mean_df.groupby('sample_type'):
        x = sub['w'].values
        y = sub['C0_mean'].values
        A = np.vstack(( x**2, x, np.ones_like(x) )).T
        # Choose bounds per sample_type
        if stype in ("LR", "LT-LW"):
            lower = [-np.inf, 0.0, 0.0]
            upper = [0.0, np.inf, np.inf]
        else:
            lower = [0.0, 0.0, 0.0]
            upper = [np.inf, np.inf, np.inf]
        # Solve
        res = lsq_linear(A, y, bounds=(lower, upper))
        a, b, c = res.x
        el_fits[stype] = (a, b, c)

    return el_fits
# end: fit_elastic_comp

# Calculate elastic strain
def calculate_elastic_strain(comp_el, sample_type, stress, wdata, load=True):

    # Initialize array
    N = len(wdata)
    eps_el = [0.0] * N

    if not load:
        # All zeros if not loading
        return eps_el

    # Extract the three coefficients for compliance: a, b, c
    a, b, c = comp_el[sample_type]

    for i in range(N):
        # Compute compliance at wdata[i]
        C_i = a * wdata[i]**2 + b * wdata[i] + c
        # Instantaneous elastic strain = compliance * stress
        eps_el[i] = C_i * stress[i]

    return eps_el
# end: def calculate_elastic_strain

# Pick local strain minimum closest to end cycle
def snap_minima_to_cycles(min_indices, new_cycle_idx, *, max_gap=None, unique=True):
    
    min_indices = np.asarray(min_indices)
    anchors = np.asarray(new_cycle_idx)

    if min_indices.size == 0 or anchors.size == 0:
        return np.array([], dtype=int), np.array([], dtype=int), np.array([], dtype=int)

    # Work on a sorted copy of minima for fast nearest-neighbor via searchsorted
    mins_sorted = np.sort(np.unique(min_indices))
    pos = np.searchsorted(mins_sorted, anchors)

    left_idx = np.clip(pos - 1, 0, len(mins_sorted) - 1)
    right_idx = np.clip(pos,       0, len(mins_sorted) - 1)

    left_vals = mins_sorted[left_idx]
    right_vals = mins_sorted[right_idx]

    # Choose the closer side
    choose_right = np.abs(right_vals - anchors) < np.abs(left_vals - anchors)
    nearest = np.where(choose_right, right_vals, left_vals)
    dist = np.abs(nearest - anchors)

    # Apply max_gap filter if requested
    keep = np.ones(len(anchors), dtype=bool)
    if max_gap is not None:
        keep &= (dist <= max_gap)

    nearest = nearest[keep]
    anchors_kept = anchors[keep]
    dist = dist[keep]

    if not unique or nearest.size == 0:
        return nearest, anchors_kept, dist

    # Resolve duplicates: keep the anchor with smallest distance per chosen minimum
    # (ties resolved by first occurrence)
    order = np.lexsort((np.arange(len(nearest)), dist))  # sort by distance, then stable index
    nearest_sorted = nearest[order]
    anchors_sorted = anchors_kept[order]
    dist_sorted = dist[order]

    # Keep first occurrence of each chosen minimum
    uniq_mask = np.ones_like(nearest_sorted, dtype=bool)
    uniq_mask[1:] = nearest_sorted[1:] != nearest_sorted[:-1]

    picked_minima = nearest_sorted[uniq_mask]
    picked_anchors = anchors_sorted[uniq_mask]
    picked_dist = dist_sorted[uniq_mask]

    # (Optional) restore original anchor order
    sort_back = np.argsort(np.argsort(picked_anchors))
    return picked_minima[sort_back], picked_anchors[sort_back], picked_dist[sort_back]
# end: def snap_minima_to_cycles

# Fit viscoelastic compliances (prony coeff.) to moisture
def fit_viscoelastic_comp(viscoel_path):

    # Read csv file
    df = pd.read_csv(viscoel_path, sep=",", dtype={"sample_type": str, "RH": float, "avg_comp_i": str})
    # Parse string‐list into float-list
    df["comp_i"] = df["avg_comp_i"].apply(lambda s: [float(x) for x in ast.literal_eval(s)])
    # Calculate average w for each RH
    df["w"] = RH_to_w_mean(df["RH"]) 
    # Fit quadratic function in w
    ve_fits = {}
    for stype, sub in df.groupby("sample_type"):
        w_vals = sub["w"].values
        C = np.vstack(sub["comp_i"].values)
        polys = [np.polyfit(w_vals, C[:, j], 2) for j in range(C.shape[1])]
        ve_fits[stype] = polys

    return ve_fits
# end: def fit_viscoelastic_comp

# Calculate prony series
def prony_response(comp_i, tdata, tau_0):
    return (np.sum(comp_i) - np.sum(comp_i[:, None] * np.exp(-tdata / tau_0[:, None]), axis=0))
# end: def prony_response

# Calculate prony coefficients at w
def comp_i_at_w(comp_ve, stype, wval):
    polys = comp_ve[stype]
    return np.array([ np.polyval(p, wval) for p in polys ])
# end: def comp_i_at_w

# Calculate viscoelastic creep compliance at given time and w
def compliance_at_t_w(comp_ve, stype, tdata, wval, tau_0):
    c_i = comp_i_at_w(comp_ve, stype, wval)
    return prony_response(c_i, tdata, tau_0)
# end: def compliance_at_t_w

# Calculate viscoelastic strain
def calculate_viscoelastic_strain(comp_ve, sample_type, stress, tdata, wdata, eps_el0):

    # Set retardation times [h]
    tau_0 = np.array([0.1, 1., 10., 100.])
    # Calculate viscoelastic strain
    eps_ve = []
    eps_i = np.zeros(tau_0.shape)
    eps_ve.append(eps_el0)
    for i in range(1, len(tdata)):
        # Calculate viscoelastic strain components
        comp_i_n1 = comp_i_at_w(comp_ve, sample_type, wdata[i])
        comp_i_n0 = comp_i_at_w(comp_ve, sample_type, wdata[i-1])
        comp_i = (comp_i_n1 + comp_i_n0) /2.
        deps_ve = np.zeros(tau_0.shape)
        deps_ve = 1. / tau_0 * (comp_i * stress[i] - eps_i)
        eps_i += deps_ve * (tdata[i]-tdata[i-1])
        eps_ve.append(np.sum(eps_i)+eps_el0)

    return eps_ve
# end: def calculate_viscoelastic_strain

# Calculate reference creep curves
def calculate_ref_viscoel_curves(comp_ve, sample_type, stress, tdata, wdata, eps_el0):

    # Set retardation times [h]
    tau_0 = np.array([0.1, 1., 10., 100.])
    # Calculate ref. curve at 30% RH = 0.07 mc
    eps_ve_ref30 = compliance_at_t_w(comp_ve, sample_type, tdata, min(wdata), tau_0) * stress + eps_el0
    # Calculate ref. curve at 90% RH = 0.20 mc
    eps_ve_ref90 = compliance_at_t_w(comp_ve, sample_type, tdata, max(wdata), tau_0) * stress + eps_el0

    return eps_ve_ref30, eps_ve_ref90
# end: def calculate_ref_viscoel_curves

# Fit monotonic regression
def unimodal_fit(x, y):

    # Initializations       
    x = np.asarray(x)
    y = np.asarray(y)
    N = len(y)
    best_err = np.inf
    best_fit = None

    # Try each possible peak-location k
    for k in range(1, N-1):
        # Rising isotonic on [0..k]
        ir_inc = IsotonicRegression(increasing=True)
        y_inc = ir_inc.fit_transform(x[:k+1], y[:k+1])

        # Falling  isotonic on [k..N-1]
        ir_dec = IsotonicRegression(increasing=False)
        y_dec = ir_dec.fit_transform(x[k:], y[k:])

        # Stitch (avoid doubling the k-th point)
        y_fit = np.concatenate([y_inc[:-1], y_dec])
        err = np.sum((y - y_fit)**2)
        if err < best_err:
            best_err  = err
            best_fit  = y_fit

    return best_fit
# end: def unimodal_fit

# Overlap and fit cycles
def analyze_cycles(tdata, wdata, edata, w_cycle_idx, eps_cycle_idx, firstc=3, lastc=7, num_grid_points=1000):

    # Convert to arrays
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    edata = np.asarray(edata)

    # Compute relative indices and segment boundaries
    firstc = firstc-2
    if lastc!=-1:
        lastc = lastc - 1
        w_cycle_idx = w_cycle_idx[firstc:lastc+1]
        eps_cycle_idx = eps_cycle_idx[firstc:lastc+1]
    else:
        w_cycle_idx = w_cycle_idx[firstc:]
        eps_cycle_idx = eps_cycle_idx[firstc:]
    w_segments = list(zip(w_cycle_idx, w_cycle_idx[1:]))
    eps_segments = list(zip(eps_cycle_idx, eps_cycle_idx[1:]))

    # Collect and shift cycles
    cycles = []
    for (w_start, w_end), (eps_start, eps_end) in zip(w_segments, eps_segments):
        # W segment and its relative time (0..1)
        t_w = tdata[w_start+1:w_end+1]
        w_seg = wdata[w_start+1:w_end+1]
        t_w_rel = (t_w - t_w[0]) / (t_w[-1] - t_w[0])
        # E segment and its relative time (0..1)
        t_e = tdata[eps_start+1:eps_end+1]
        e_seg = edata[eps_start+1:eps_end+1]
        t_e_rel = (t_e - t_e[0]) / (t_e[-1] - t_e[0])
        # Interpolate strain on the w grid
        e_on_w = np.interp(t_w_rel, t_e_rel, e_seg)
        # Locate peak
        peak_i = np.argmax(e_on_w)
        peak_val = e_on_w[peak_i]
        # Store cycle data
        cycles.append({
            't_seg_rel': t_w_rel,
            'w':       w_seg,
            'eps':       e_on_w,
            'last_eps':  e_on_w[-1],
            'peak_eps':  peak_val})

    ######## BUILD TEMPLATE FOR ALL CYCLES (except 1) ########
    # Use first cycle's peak as reference
    ref_peak = cycles[0]['peak_eps']
    # Apply vertical shift based on peaks
    for c in cycles: c['eps_shifted'] = c['eps'] - c['peak_eps'] + ref_peak
    # Build common time grid where all shifted cycles overlap
    t_fit = np.linspace(0, 1, num_grid_points)

    # Interpolate each shifted cycle onto the common grid
    s_interp = np.vstack([c['eps_shifted'] for c in cycles])
    avg_s_time = np.mean(s_interp, axis=0)
    avg_s_time = avg_s_time - avg_s_time[0]
    s_interp = np.vstack([UnivariateSpline(c['t_seg_rel'], c['eps_shifted'], k=5, s=0.00001)(t_fit) for c in cycles])
    # Calculate average strain
    avg_s_time = np.mean(s_interp, axis=0)
    # Shift template to start from second cycle
    shift = avg_s_time[0] - edata[eps_cycle_idx[0]+1]
    avg_s_time = avg_s_time - shift

    # Try to fit avoiding any possible final increase
    if lastc != -1:       
        # Fit template
        y_fit = unimodal_fit(t_fit, avg_s_time)
        x = t_fit
        peak_idx = np.argmax(y_fit)
        # Fit a strictly‐decreasing isotonic to the tail
        ir_down = IsotonicRegression(increasing=False)
        x_tail = x[peak_idx:]
        y_tail = y_fit[peak_idx:]
        y_tail_iso = ir_down.fit_transform(x_tail, y_tail)
        # Stitch it back together
        avg_s_time = np.concatenate([y_fit[:peak_idx], y_tail_iso])


    ######## BUILD TEMPLATE FOR 1st CYCLE ########
    # Compute delta sorption
    w0 = wdata[:w_cycle_idx[0]+1]
    w1 = wdata[w_cycle_idx[0]+1:w_cycle_idx[1]+1]
    idx1 = w1.argmax()
    shift = w1[idx1] - w0[idx1]
    w1 = w1 - shift
    delta_w = [x/y for x, y in zip(w0, w1)]

    # Pick target times
    t_rel = (tdata[w_cycle_idx[0]+1:w_cycle_idx[1]+1] - tdata[w_cycle_idx[0]+1]) / (tdata[w_cycle_idx[1]+1] - tdata[w_cycle_idx[0]+1])
    t0_rel = t_rel[idx1]
    idx_fit = np.searchsorted(t_fit, t0_rel)
    slice_times = t_fit[:idx_fit+1]
    # Calculate scaling factor by interpolating delta_w
    scale = np.interp(slice_times, t_rel[:idx1+1], delta_w[:idx1+1])
    # Scale strain  proportional to sorption
    avg_s_time0 = np.concatenate((avg_s_time[:idx_fit+1] * scale, avg_s_time[idx_fit+1:]), axis=0)
    # Shift template I to start from elastic strain
    shift = avg_s_time0[0] - edata[0]
    avg_s_time0 = avg_s_time0 - shift

    # Shift template II to start from 0
    shift = avg_s_time[0]
    avg_s_time = avg_s_time - shift
    
    return [t_fit, avg_s_time0, avg_s_time]
# end: def analyze_cycles

# Calculate mechanosorptive strain from incremental scheme
def calculate_mechanosorptive_strain_from_inc(wdata, tdata, sdata, edata, w_cycle_idx, eps_cycle_idx, firstc_l=6, lastc_l=9, firstc_un=None, lastc_un=-1):
    
    #print(w_cycle_idx, len(w_cycle_idx), eps_cycle_idx, len(eps_cycle_idx))
    ########## Calculate strain increments ##########
    # Find strain templates
    [t_cycle, eps_cycle, eps_cycle1] = analyze_cycles(tdata, wdata, edata, w_cycle_idx, eps_cycle_idx)#, firstc=firstc_l, lastc=lastc_l)

    # Initialize arrays
    seg_times = []
    seg_vals = []
    seg_moist = []
    seg_stress = []
    
    for i, end in enumerate(w_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else w_cycle_idx[i-1]
        # Extract time
        t_w = tdata[start+1:end+1]
        t_w_rel = (t_w - t_w[0]) / (t_w[-1] - t_w[0])
        # Extract moisture
        w_w = wdata[start+1:end+1]
        w_marks = UnivariateSpline(t_w_rel, w_w, k=5, s=0)(t_cycle)
        seg_moist.append(w_marks)
        # Extract stress
        s_w = sdata[start+1:end+1]
        s_marks = UnivariateSpline(t_w_rel, s_w, k=5, s=0)(t_cycle)
        seg_stress.append(s_marks)
    
    for i, end in enumerate(eps_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else eps_cycle_idx[i-1]
        # Extract time
        t_eps = tdata[start+1:end+1]
        t_eps_rel = (t_eps - t_eps[0]) / (t_eps[-1] - t_eps[0])
        # Extract strain
        e_eps = edata[start+1:end+1]
        e_marks = UnivariateSpline(t_eps_rel, e_eps, k=5, s=0)(t_cycle)
        seg_vals.append(e_marks)

    ########## Correct jump of template I (1st cycle) ##########
    # Get corresponding moisture and time
    w_end = seg_moist[0][-1]
    w_temp = seg_moist[0][:len(seg_moist[0])//2]
    idx = np.abs(w_temp - w_end).argmin()
    t_start = t_cycle[idx]
    idx_t = np.abs(t_cycle - t_start).argmin()
    # Find delta strain
    delta_end = eps_cycle[-1] - eps_cycle[idx_t]
    x1, y1 = t_cycle[0], 0
    x2, y2 = t_cycle[-1], delta_end
    m0 = (y2 - y1) / (x2 - x1)
    b0 = y1 - m0 * x1
    # Shift jump by linear scaling
    shift_cycle = m0*t_cycle + b0 
    eps_cycle = eps_cycle - shift_cycle
    ########## Calculate mechanosorp. strain of 1st cycle ##########
    # Build temporary strain of first cycle
    shift = eps_cycle[0] - edata[0] # shift to start at initial elastic strain (if not done before)
    first_cycle = eps_cycle - shift
    first_cycle_ms = seg_vals[0] - first_cycle

    ########## Correct jump of template II (from 2nd cycle) ##########
    # Find delta strain
    delta_end = eps_cycle1[-1] - eps_cycle1[0]
    x1, y1 = t_cycle[0], eps_cycle1[0] - eps_cycle1[0]
    x2, y2 = t_cycle[-1], delta_end
    m1 = (y2 - y1) / (x2 - x1)
    b1 = y1 - m1 * x1
    # Shift jump by linear scaling
    shift_cycle = m1*t_cycle + b1
    eps_cycle1 = eps_cycle1 - shift_cycle

    ######### Calculate mechanosorp. strain of 1st cycle ##########
    second_cycle = eps_cycle1 - eps_cycle1[0] # shift to start at 0 (if not done before)
    seg_eps = []
    seg_eps.append(first_cycle_ms)
    for i in range(1,len(w_cycle_idx)):
        #print(len(seg_vals[i]), len(second_cycle), seg_vals[i][0])
        eps_inc = seg_vals[i] - (second_cycle + seg_vals[i][0]) + seg_eps[-1][-1]
        seg_eps.append(eps_inc)
    
    """plt.figure(figsize=(7,5))
    plt.plot(t_cycle, first_cycle_ms, label="mcs1")
    plt.plot(t_cycle, eps_cycle, label="Theta 1")
    plt.plot(t_cycle, seg_vals[0], label="eps_red")
    # Extract time
    t_eps = tdata[:eps_cycle_idx[0]+1]
    t_eps_rel = (t_eps - t_eps[0]) / (t_eps[-1] - t_eps[0])
    # Extract strain
    e_eps = edata[:eps_cycle_idx[0]+1]
    plt.plot(t_eps_rel, e_eps, label="eps_red_0")
    #plt.plot(t_cycle, eps_cycle1, label="Theta 2")
    plt.xlabel("t [s]")
    plt.ylabel("Strain ε [-]")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()"""


    """if firstc_un is not None:
        ########## Find stress drop ##########
        # Find stress drop
        stress_flat = np.concatenate(seg_stress)
        drop_idx_flat = find_local_drop(stress_flat)
        # Fit unloaded cycles
        _, [t_end, eps_end] = analyze_cycles(tdata, wdata, edata, cycle_idx, firstc=firstc_un, lastc=lastc_un)
        ########## Correct jump of template III (last cycle) ##########
        delta_end = eps_end[-1] - eps_end[0]
        x1, y1 = t_cycle[0], eps_end[0] - eps_end[0]
        x2, y2 = t_cycle[-1], delta_end
        m = (y2 - y1) / (x2 - x1)
        b = y1 - m * x1
        shift_cycle = m*t_cycle + b 
        eps_end = eps_end - shift_cycle
        ########## Correct strain for unloading ##########
        # Build temporary strain of last cycle
        last_cycle = UnivariateSpline(t_end, eps_end, k=5, s=0)(seg_times[0])
        shift = last_cycle[0]
        last_cycle = last_cycle - shift

        M = seg_stress[0].shape[0]
        drop_cycle = drop_idx_flat // M
        inc_vals = []
        inc_vals.append(first_cycle_ms)
        for i,d in enumerate(cycle_idx):
            if i > 0:
                # Evaluate cycle at given segmented times
                idx = np.searchsorted(tdata, seg_times[i], side='left')
                idx = np.clip(idx, 0, len(tdata)-1)
                prev = np.clip(idx-1, 0, len(tdata)-1)
                choose_prev = np.abs(tdata[prev] - seg_times[0]) <= np.abs(tdata[idx] - seg_times[0])
                idx[choose_prev] = prev[choose_prev]
                epsP = edata[idx]
                
                if i > drop_cycle:
                    shift = last_cycle[0] - epsP[0]
                    inc_vals.append(epsP - last_cycle + shift + inc_vals[-1][-1])

                else:
                    shift = second_cycle[0] - epsP[0]
                    inc_vals.append(epsP - second_cycle + shift + inc_vals[-1][-1])

        # Flatten strain array        
        eps_flat  = np.concatenate(inc_vals)"""

    # Re-sample time array
    for i, end in enumerate(eps_cycle_idx):
        # Set previous cycle
        start = -1 if i == 0 else eps_cycle_idx[i-1]
        # Extract time
        t0 = tdata[start+1]
        t1 = tdata[end]
        seg_times.append(t_cycle * (t1 - t0) + t0)

    return first_cycle, second_cycle, second_cycle, seg_eps, seg_times, seg_moist, seg_stress
# end: def calculate_mechanosorptive_strain_from_inc

# Fitting model of mechanosorptive strain
def mechanosorptive_model(comp_j, stress, tdata, wdata):
    
    # Charachteristic moistures [-]
    mu_0 = np.array([1., 10., 100.])/100.
    # Build moisture‐rate array
    tdata = np.asarray(tdata)
    wdata = np.asarray(wdata)
    dt = np.diff(tdata)
    dw = np.diff(wdata)
    wrate = np.concatenate(([0.], np.abs(dw) / dt))

    #print(tdata, wrate, comp_j)
    
    # Initialize
    eps_j  = np.zeros_like(mu_0)
    eps_ms = np.zeros_like(wrate)
    
    # Calculate mechanosorptive strain
    for i in range(1, len(tdata)):
        delta_t = tdata[i] - tdata[i-1]
        # Calculate mechanosorptive increment
        deps = (wrate[i] / mu_0) * (comp_j * stress[i] - eps_j)
        eps_j  += deps * delta_t
        eps_ms[i] = eps_j.sum()

    #print(eps_ms)
    
    return eps_ms
# end: def mechanosorptive_model

# Plot mechanosorptive strain vs moisture
def calculate_mechanosorptive_strain(sample_type, w_arr, t_arr, s_arr, eps_arr, w_cycle_idx, eps_cycle_idx):

    # Calculate strain components from total strain
    eps_comp = calculate_strain_components(sample_type, w_arr, t_arr, s_arr, eps_arr)
    ref_strain = eps_arr - eps_comp[5] # ref. strain = total - viscoel.
    
    """plt.figure(figsize=(7,5))
    plt.plot(eps_comp[0], eps_arr, label="mcs1")
    plt.plot(eps_comp[0], ref_strain, label="eps_red")
    plt.xlabel("t [s]")
    plt.ylabel("Strain ε [-]")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()"""

    # Calculate mechanosorptive strain from incremental scheme
    first_cycle, second_cycle , _, seg_eps, seg_times, seg_moist, seg_stress = calculate_mechanosorptive_strain_from_inc(eps_comp[1], eps_comp[0], eps_comp[2], ref_strain, w_cycle_idx, eps_cycle_idx)
    exp_eps_ms  = np.concatenate(seg_eps)
    times_flat  = np.concatenate(seg_times)
    w_flat = np.concatenate(seg_moist)
    s_flat = np.concatenate(seg_stress)

    """plt.figure(figsize=(8,8))
    plt.plot(times_flat, exp_eps_ms, label="eps_ms")
    plt.plot(eps_comp[0], ref_strain, label="eps_red")
    
    for i in range(len(eps_cycle_idx)):
        if i == 0:
            plt.plot(seg_times[0], first_cycle, label="template", color='green')
        else:
            plt.plot(seg_times[i], second_cycle+ref_strain[eps_cycle_idx[i-1]], color='green')
    
    
    for idx in eps_cycle_idx:
        plt.axvline(x=t_arr[idx], color="grey", linestyle="--", alpha=0.7)

    plt.xlabel("t [s]")
    plt.ylabel("Strain [-]")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()"""

    # Fit incremental mechanosorptive strain
    initial_guess = np.array([0.01]*3, dtype=float) # initial guess
    # Residual function for least_squares
    def residual(comp_j):
        pred = mechanosorptive_model(comp_j, s_flat, times_flat, w_flat)
        return (pred - exp_eps_ms)
    # Bound compliances to be non‐negative
    result = least_squares(residual, initial_guess, bounds=(0, np.inf), xtol=1e-8, ftol=1e-8)
    comp_opt = result.x # get prony coefficients
    eps_ms_fit = mechanosorptive_model(comp_opt, s_flat, times_flat, w_flat) # fitted strain

    return times_flat, w_flat, s_flat, exp_eps_ms, eps_ms_fit, comp_opt
# end: def plot_mechanos_strain

### 3) Prepare experimental datasets

In [ ]:
################## PREPARE EXPERIMENTAL DATASETS ##################
# Load data from dvs test
data = np.load(dvs_path)
wdata0 = data['w'][0] # moisture of 1st cycle
tdata0 = data['t'][0] # time of 1st cycle
wdata = data['w'][1] # moisture of following cycles
tdata = data['t'][1]  # time of following cycles


# Retrieve sample types
sample_types = get_sample_types(folder_path)[3:]  # keep RL, RT, TR (per your earlier setup)

# Extract datasets
dataset = []
for sample_type in sample_types:

    # Find experiments of sample type
    matches = get_samples_by_type(sample_type, folder_path)

    # Group by loading degree
    grouped = defaultdict(list)
    for exp, exp_path, sample_name in matches:
        ld = np.round(extract_loading_deg(exp_path, sample_name), 1)
        grouped[ld].append((exp, exp_path, sample_name))

    # Build dataset entries for each loading degree and sample
    for ld, group in grouped.items():
        for idx, (exp, exp_path, sample_name) in enumerate(group):

            # Prepare file paths and load raw data
            file_folder = os.path.join(exp_path, "Results", sample_name)
            strain_results = load_strain_struct(file_folder)

            total_strain = strain_results['axial_strain']
            time = strain_results['exp_duration']
            stress = strain_results['stress_meas']

            # Cut off pre-mechanosorption
            cycle_idx = find_cycle_ends(strain_results['RH_mean'], tol=1, merge_gap=5)  # find cycle ends
            t0 = cycle_idx[0]  # get first index = creep pre-mechanosorption
            time = time[t0:] - time[t0]
            stress = stress[t0:]
            total_strain = total_strain[t0:] - total_strain[t0]
            cycle_idx = cycle_idx[1:] - t0

            # Determine load drop
            num_cycles = len(cycle_idx)
            if num_cycles > 8:
                # Drop unloading cycles
                guess_drop_idx = (cycle_idx[-2] + cycle_idx[-4]) / 2.0
                drop_idx = find_local_drop(stress, stress_drop=guess_drop_idx, window=10)
                idx_before, drop_idx, idx_after = locate_drop_idx(cycle_idx, drop_idx)
                t_full, w_full, s_full = build_dynamic_cycles(
                    tdata0, wdata0, tdata, wdata,
                    time, cycle_idx, idx_before, drop_idx, [stress[0], stress[-1]]
                )
            else:
                idx_before = drop_idx = idx_after = len(time) - 1
                t_full, w_full, s_full = build_dynamic_cycles(
                    tdata0, wdata0, tdata, wdata,
                    time, cycle_idx, idx_before, drop_idx, [stress[0]]
                )

            # Find new cycles index
            n_pre = np.searchsorted(cycle_idx, idx_before, side="right")
            cycle_len = len(tdata)
            new_cycle_idx = [int((i + 1) * cycle_len - 1) for i in range(n_pre)]
            if (sample_type == 'RT' and ld == 0.5 and idx == 4) or (sample_type == 'RT' and ld == 0.3 and idx in [1, 2, 3]):
                new_cycle_idx = np.concatenate([new_cycle_idx[:-1], [len(t_full) - 1]])
            else:
                new_cycle_idx = np.concatenate([new_cycle_idx, [len(t_full) - 1]])

            # Up-sample strain with a spline
            spline = make_interp_spline(time, total_strain, k=5)
            eps_full = spline(t_full)

            # Find index of local minimum strains
            min_indices, _ = find_peaks(-eps_full)
            picked_minima, picked_anchors, distances = snap_minima_to_cycles(
                min_indices, new_cycle_idx, max_gap=None, unique=True
            )
            picked_minima = np.unique(picked_minima)

            # Cut last cycle out?
            t_sel = np.asarray(t_full)[new_cycle_idx]
            diffs = np.diff(t_sel)
            prev = diffs[:-1]
            last = diffs[-1]
            baseline = np.median(prev) if len(prev) else last
            tol = 0.5
            if abs(last - baseline) > tol and len(new_cycle_idx) > 0:
                new_cycle_idx = new_cycle_idx[:-1]

            if len(picked_minima) < len(new_cycle_idx):
                picked_minima = np.append(picked_minima, new_cycle_idx[-1])
            picked_minima = np.sort(picked_minima)

            # Add initial elastic strain
            comp_el = fit_elastic_comp(elastic_path)
            eps_el = calculate_elastic_strain(comp_el, sample_type, s_full, w_full)
            eps_full = eps_full + eps_el[0]

            # === calculate mechanosorptive strain and compliance values ===
            t_flat, w_flat, s_flat, exp_eps_ms, eps_ms_fit, comp_opt = calculate_mechanosorptive_strain(
                sample_type, w_full, t_full, s_full, eps_full, new_cycle_idx, picked_minima)

            # ---- store only the metadata we need for Excel (so we don't have to reload later)
            meta = {
                'thickness'   : float(strain_results.get('thickness')),
                'width'       : float(strain_results.get('width')),
                'area'        : float(strain_results.get('area')),
                'exp_duration': max(t_full),
                'load_nom'    : float(strain_results.get('load_nom')),
                'stress_nom'  : strain_results.get('load_nom')/strain_results.get('area'),
                'load_meas'   : s_full[0] * strain_results.get('area'), #strain_results.get('load_meas'),
                'stress_meas' : s_full[0], #strain_results.get('stress_meas'),
                'loading_deg' : float(strain_results.get('loading_deg')),
                'comp_opt'    : comp_opt,
            }

            # Append to dataset
            dataset.append({
                'exp'            : exp,
                'sample_type'    : sample_type,
                'sample_name'    : sample_name,
                'ld'             : ld,
                't_full'         : t_flat,
                'w_full'         : w_flat,
                's_full'         : s_flat,
                'eps_full'       : exp_eps_ms,
                'w_cycle_idx'    : new_cycle_idx,
                'eps_cycle_idx'  : picked_minima,
                'stress'         : stress,
                'meta'           : meta,  # <— store meta for Excel writing
            })


### 4) Save excel files

In [ ]:
os.makedirs(savepath, exist_ok=True)
# ======================= EXCEL HELPERS =======================
def _sanitize_sheet_name(name: str) -> str:
    """Make a valid Excel sheet name (<=31 chars and no \ / * ? : [ ])"""
    name = re.sub(r'[\\/*?:\[\]]', '_', str(name))
    return name[:31] if len(name) > 31 else name

def _dedupe(name: str, used: set) -> str:
    """Ensure sheet name uniqueness within a workbook."""
    base = name
    i = 1
    while name in used:
        suffix = f"_{i}"
        name = (base[: 31 - len(suffix)]) + suffix
        i += 1
    used.add(name)
    return name

def _meta_dataframe(entry: dict) -> pd.DataFrame:
    """Top two-column block as specified."""
    m = entry['meta']
    comp_opt = m.get('comp_opt', [np.nan, np.nan, np.nan])
    # Fixed Mu values
    mu_vals = [1, 10, 100]

    rows = [
        ("Experiment",                  entry['exp']),
        ("Sample",                      entry['sample_name']),
        ("Sample Type",                 entry['sample_type']),
        ("Thickness [mm]",              m.get('thickness')),
        ("Width [mm]",                  m.get('width')),
        ("Area [mm^2]",                 m.get('area')),
        ("Duration [h]",                m.get('exp_duration')),
        ("Nominal Load [N]",            m.get('load_nom')),
        ("Nominal Stress [MPa]",        m.get('stress_nom')),
        ("Creep Load [N]",              m.get('load_meas')),
        ("Creep Stress [MPa]",          m.get('stress_meas')),
        ("Loading Degree [%]",  m.get('loading_deg') * 100),
        ("Compliance_1 [1/GPa]",        comp_opt[0]*1000 if len(comp_opt) > 0 else np.nan),
        ("Compliance_2 [1/GPa]",        comp_opt[1]*1000 if len(comp_opt) > 0 else np.nan),
        ("Compliance_3 [1/GPa]",        comp_opt[2]*1000 if len(comp_opt) > 0 else np.nan),
        ("Mu_1 [%]",                    mu_vals[0]),
        ("Mu_2 [%]",                    mu_vals[1]),
        ("Mu_3 [%]",                    mu_vals[2]),
    ]
    return pd.DataFrame(rows, columns=["", ""])

# ======================= EXCEL WRITER =======================
def save_excel_datasets(dataset: list, savepath: str):
    # Group by sample_type
    by_type = defaultdict(list)
    for entry in dataset:
        by_type[entry['sample_type']].append(entry)

    for sample_type, entries in by_type.items():
        out_file = os.path.join(savepath, f"{sample_type}_samples_results.xlsx")
        used_sheet_names = set()

        with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
            for entry in entries:
                # Top meta block
                meta_df = _meta_dataframe(entry)

                # Bottom table
                t_full   = np.asarray(entry['t_full'])
                w_full   = np.asarray(entry['w_full'])
                s_full   = np.asarray(entry['s_full'])
                eps_full = np.asarray(entry['eps_full'])

                with np.errstate(divide='ignore', invalid='ignore'):
                    creep_comp = np.where(s_full != 0, (eps_full / s_full) * 1000.0, np.nan)

                data_df = pd.DataFrame({
                    "Time [h]":                  t_full,
                    "Moisture [%]":              w_full*100,
                    #"Stress [MPa]":              s_full,
                    "Creep Strain [-]":          eps_full,
                    "Creep Compliance [1/GPa]":  creep_comp,
                })

                # Sheet name = sample_name (sanitized & de-duplicated)
                sheet_name = _dedupe(_sanitize_sheet_name(entry['sample_name']), used_sheet_names)

                # Write meta at the top (A1), no headers
                meta_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=0, startcol=0)

                # Leave one empty row, then write data table with headers
                start_row = len(meta_df) + 2
                data_df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=start_row, startcol=0)

                # Basic column widths (openpyxl)
                ws = writer.sheets[sheet_name]
                ws.column_dimensions['A'].width = 22  # labels
                ws.column_dimensions['B'].width = 28  # values
                # Ensure data columns are readable
                for col in ['A', 'B', 'C', 'D', 'E']:
                    ws.column_dimensions[col].width = max(ws.column_dimensions[col].width or 0, 18)

        print(f"Saved: {out_file}")


# ====== USAGE ======
# After your dataset-building loop finishes, just call:
save_excel_datasets(dataset, savepath)
